**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install captum
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import matplotlib.pyplot as plt
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import numpy as np
import torch

import torchvision
from torchvision import models
from torchvision import transforms

from captum.attr import IntegratedGradients, Saliency, GuidedBackprop
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import visualization as viz

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract, download_files_maybe_extract
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

download_files_maybe_extract([
    DATA_HOME.format("imagenet_classes"),
    DATA_HOME.format("images/toucan.png"),
    DATA_HOME.format("images/raccoon_example.jpg"),
    DATA_HOME.format("images/nails.jpg"),
    DATA_HOME.format("images/screws.jpg"),
    DATA_HOME.format("images/ambulance.jpg")
], directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }
with open("data/imagenet_classes", "r") as file:
    class_names = [c[:-1] for c in file.readlines()]

def resize_image(img, size=224):
    transform = transforms.Compose([
        transforms.Resize(size)
    ])
    return np.asarray(transform(img))

def decode_proba(proba, top=5, verbose=True):
    with torch.no_grad():
        probs, classes = torch.topk(proba, 5)
        probs = probs[0]
        classes = classes[0]

    if verbose:
        for p, c in zip(probs.cpu().numpy(),
                        classes.cpu().numpy()):
            print("{}:\t{} ({})".format(
                np.array2string(p, precision=5),
                class_names[c], c))
            
    return probs, classes

vis_kwargs_default = dict(
    methods=['original_image', 'alpha_scaling', 'heat_map'],
    cmap="viridis",
    show_colorbar=True,
    signs=['all', 'absolute_value', 'absolute_value'],
    fig_size=(12.5, 4)
)

class ProcImage:
    def __init__(self, model, weights, device, verbose=True):
        self.model = model
        self.device = device
        self.verbose = verbose

        weight_transforms = weights.transforms()
        self.normalize = transforms.Normalize(
            mean=weight_transforms.mean,
            std=weight_transforms.std
        )

    def preproc_image(self, img, size=224):
        if not isinstance(img, Image.Image):
            img = torchvision.transforms.functional.to_pil_image(img)

        img = img.convert('RGB')
        transform = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            self.normalize
        ])

        tensor = transform(img).unsqueeze(0)
        return tensor

    def __call__(self, img_path):
        img = Image.open(img_path)
        img_preproc = self.preproc_image(img)
        img_resized = resize_image(img)
        
        if not self.device is None:
            img_preproc = img_preproc.to(self.device)
            self.model.to(self.device)

        self.model.eval()
        img_preproc.requires_grad = True
        
        output = torch.nn.functional.softmax(self.model(img_preproc), dim=1)
        pred_class = decode_proba(output, verbose=True)[1][0]

        return img_preproc, img_resized, pred_class

def visualize_attrib(attrib, **kwargs):
    kwargs_merged = dict(**vis_kwargs_default)
    kwargs_merged.update(kwargs)
    attrib = np.transpose(attrib.squeeze().cpu().detach().numpy(), (1,2,0))

    # workaround for a matplotlib-related bug in captum
    visualize_image_attr_multiple(attrib, img_resized, **kwargs_merged)
    #viz.visualize_image_attr_multiple(attrib, img_resized, **kwargs_merged)

# workaround for a matplotlib-related bug in captum

from matplotlib.figure import Figure
from captum.attr._utils.visualization import (
    _prepare_image, ImageVisualizationMethod, _normalize_attr,
    VisualizeSign, make_axes_locatable, LinearSegmentedColormap
)

def visualize_image_attr(
    attr, original_image=None, method="heat_map", sign="absolute_value",
    plt_fig_axis=None, outlier_perc=2, cmap=None, alpha_overlay=0.5,
    show_colorbar=False, title=None, fig_size=(6, 6), use_pyplot=True
):
    # Create plot if figure, axis not provided
    if plt_fig_axis is not None:
        plt_fig, plt_axis = plt_fig_axis
    else:
        if use_pyplot:
            plt_fig, plt_axis = plt.subplots(figsize=fig_size)
        else:
            plt_fig = Figure(figsize=fig_size)
            plt_axis = plt_fig.subplots()

    if original_image is not None:
        if np.max(original_image) <= 1.0:
            original_image = _prepare_image(original_image * 255)
    elif ImageVisualizationMethod[method] != ImageVisualizationMethod.heat_map:
        raise ValueError(
            "Original Image must be provided for"
            "any visualization other than heatmap."
        )

    # Remove ticks and tick labels from plot.
    plt_axis.xaxis.set_ticks_position("none")
    plt_axis.yaxis.set_ticks_position("none")
    plt_axis.set_yticklabels([])
    plt_axis.set_xticklabels([])
    plt_axis.grid(visible=False)

    heat_map = None
    # Show original image
    if ImageVisualizationMethod[method] == ImageVisualizationMethod.original_image:
        assert (
            original_image is not None
        ), "Original image expected for original_image method."
        if len(original_image.shape) > 2 and original_image.shape[2] == 1:
            original_image = np.squeeze(original_image, axis=2)
        plt_axis.imshow(original_image)
    else:
        # Choose appropriate signed attributions and normalize.
        norm_attr = _normalize_attr(attr, sign, outlier_perc, reduction_axis=2)

        # Set default colormap and bounds based on sign.
        if VisualizeSign[sign] == VisualizeSign.all:
            default_cmap = LinearSegmentedColormap.from_list(
                "RdWhGn", ["red", "white", "green"]
            )
            vmin, vmax = -1, 1
        elif VisualizeSign[sign] == VisualizeSign.positive:
            default_cmap = "Greens"
            vmin, vmax = 0, 1
        elif VisualizeSign[sign] == VisualizeSign.negative:
            default_cmap = "Reds"
            vmin, vmax = 0, 1
        elif VisualizeSign[sign] == VisualizeSign.absolute_value:
            default_cmap = "Blues"
            vmin, vmax = 0, 1
        else:
            raise AssertionError("Visualize Sign type is not valid.")
        cmap = cmap if cmap is not None else default_cmap

        # Show appropriate image visualization.
        if ImageVisualizationMethod[method] == ImageVisualizationMethod.heat_map:
            heat_map = plt_axis.imshow(norm_attr, cmap=cmap, vmin=vmin, vmax=vmax)
        elif (
            ImageVisualizationMethod[method]
            == ImageVisualizationMethod.blended_heat_map
        ):
            assert (
                original_image is not None
            ), "Original Image expected for blended_heat_map method."
            plt_axis.imshow(np.mean(original_image, axis=2), cmap="gray")
            heat_map = plt_axis.imshow(
                norm_attr, cmap=cmap, vmin=vmin, vmax=vmax, alpha=alpha_overlay
            )
        elif ImageVisualizationMethod[method] == ImageVisualizationMethod.masked_image:
            assert VisualizeSign[sign] != VisualizeSign.all, (
                "Cannot display masked image with both positive and negative "
                "attributions, choose a different sign option."
            )
            plt_axis.imshow(
                _prepare_image(original_image * np.expand_dims(norm_attr, 2))
            )
        elif ImageVisualizationMethod[method] == ImageVisualizationMethod.alpha_scaling:
            assert VisualizeSign[sign] != VisualizeSign.all, (
                "Cannot display alpha scaling with both positive and negative "
                "attributions, choose a different sign option."
            )
            
            original_image = original_image[..., :3]
            plt_axis.imshow(
                np.concatenate(
                    [
                        original_image,
                        _prepare_image(np.expand_dims(norm_attr, 2) * 255),
                    ],
                    axis=2,
                )
            )
        else:
            raise AssertionError("Visualize Method type is not valid.")

    # Add colorbar. If given method is not a heatmap and no colormap is relevant,
    # then a colormap axis is created and hidden. This is necessary for appropriate
    # alignment when visualizing multiple plots, some with heatmaps and some
    # without.
    if show_colorbar:
        axis_separator = make_axes_locatable(plt_axis)
        colorbar_axis = axis_separator.append_axes("bottom", size="5%", pad=0.1)
        if heat_map:
            plt_fig.colorbar(heat_map, orientation="horizontal", cax=colorbar_axis)
        else:
            colorbar_axis.axis("off")
    if title:
        plt_axis.set_title(title)

    if use_pyplot:
        plt.show()

    return plt_fig, plt_axis


def visualize_image_attr_multiple(
    attr, original_image, methods, signs, titles=None,
    fig_size=(8, 6), use_pyplot=True, **kwargs
):
    assert len(methods) == len(signs), "Methods and signs array lengths must match."
    if titles is not None:
        assert len(methods) == len(titles), (
            "If titles list is given, length must " "match that of methods list."
        )
    if use_pyplot:
        plt_fig = plt.figure(figsize=fig_size)
    else:
        plt_fig = Figure(figsize=fig_size)
    plt_axis = plt_fig.subplots(1, len(methods))

    # When visualizing one
    if len(methods) == 1:
        plt_axis = [plt_axis]

    for i in range(len(methods)):
        visualize_image_attr(
            attr,
            original_image=original_image,
            method=methods[i],
            sign=signs[i],
            plt_fig_axis=(plt_fig, plt_axis[i]),
            use_pyplot=False,
            title=titles[i] if titles else None,
            **kwargs,
        )
    plt_fig.tight_layout()
    if use_pyplot:
        plt.show()
    return plt_fig, plt_axis

## Vizuálna interpretácia neurónových sietí

Neurónové siete a hlboké učenie predstavujú veľmi silnú paradigmu v oblasti strojového učenia. Nie sú však známe vysokou interpretovateľnosťou. Napriek tomu existuje niekoľko techník, ktoré dokážu poskytnúť určitú predstavu o tom, či sa neurónová sieť správa ako má. Pri tabuľkových dátach sa samozrejme vysvetlenia dajú vytvoriť pomocou metód ako LIME. Existuje však aj niekoľko techník, ktoré pracujú s obrázkami a niektoré z nich si ukážeme v tomto notebook-u.

### Načítanie modelu

Začneme načítaním predtrénovaného modelu, ktorý budeme testovať.



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights = models.ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights).to(device)
proc_image = ProcImage(model, weights, device)

### Saliency

Začneme s konceptom známym ako visual saliency (vizuálna význačnosť). Myšlienka je jednoduchá: skrátka použijeme spätné šírenie chyby (backprop, autodiff), vypočítame citlivosť predikcie na rôzne vstupné pixely a výsledok vizualizujeme: získame tzv. mapu význačností (saliency map). Zjednodušene povedané, mapa význačností rozsvieti pixely úmerne ich relevantnosti pri predikcii. To nám umožní overiť, či sieť sleduje správne časti obrázka: t.j. že predikuje lietadlo preto, že ho naozaj rozpoznáva, a nie preto, že väčšina pixelov na pozadí obrázka je modrá.



In [ ]:
attributor = Saliency(model)

Ďalej si načítame obrázok tukana, spustíme na ňom model a zobrazíme mapu význačností. Uvidíme, že oblasť okolo tukana sa rozsvieti najviac. Zvýraznené však budú aj iné oblasti. Ako čoskoro uvidíme, v tomto prípade je to skôr vlastnosťami metódy než modelu: klasický gradient nie je až takým dobrým indikátorom.



In [ ]:
img_preproc, img_resized, pred_class = proc_image('data/toucan.png')
attrib = attributor.attribute(img_preproc, target=pred_class)
visualize_attrib(attrib, outlier_perc=10)

### Navádzané spätné šírenie (guided backprop)

Druhá metóda, ktorú si vyskúšame bude produkovať omnoho lepšie výsledky. Nazýva sa navádzané spätné šírenie (guided backprop) a vlastne nie je veľmi odlišná od vizuálnej význačnosti: tiež je založená na určitej verzii gradientu. Rozdiel je v tom, že modifikuje šírenie gradientu cez ReLU aktivačné funkcie tak, že sa prešíria len nezáporné gradienty. Tým sa v podstate ignorujú signály, ktoré negatívne prispievajú k aktivácii predikovanej triedy.

Keďže sa zdá, že súčasná verzia balíka `captum` má problémy s inplace ReLU kvôli niektorým zmenám v `torch` a `torchvision` balíčkoch, model najprv prejdeme a zmeníme `.inplace` príznak všetkých ReLU na `False`.



In [ ]:
for module in model.modules():
    if isinstance(module, torch.nn.ReLU):
        module.inplace=False

In [ ]:
attributor = GuidedBackprop(model)

Teraz našu metódu opäť aplikujeme na obrázok tukana: tento raz by mal sa mali už výstupy jasne sústrediť na obrysy tukana.



In [ ]:
img_preproc, img_resized, pred_class = proc_image('data/toucan.png')
attrib = attributor.attribute(img_preproc, target=pred_class)
visualize_attrib(attrib, outlier_perc=60)

Dobre, teraz si to isté vyskúšajme aj na pár ďalších obrázkoch.



In [ ]:
for img_path in ['data/ambulance.jpg', 'data/screws.jpg', 'data/nails.jpg', 'data/raccoon_example.jpg']:
    img_preproc, img_resized, pred_class = proc_image(img_path)
    attrib = attributor.attribute(img_preproc, target=pred_class)
    visualize_attrib(attrib, outlier_perc=60)

### Prekrytie

Existuje ešte mnoho iných prístupov založených na gradiente: niektoré z nich sa dajú preskúmať s použitím dokumentácie balíčka `captum`, ktorý používame na vytvorenie vizualizácií v tomto notebook-u.

My spravíme už len jeden experiment – s vizualizačnou metódou založenou na úplne inom princípe: na prekrytí. Myšlienka je, že ak je pixel (alebo skupina pixelov) podstatný vzhľadom na predikciu, jeho prekrytie (napr. nastavením na nuly), bude mať veľmi silný dopad na predikciu. Ak teda budeme po obrázku kĺzať krycím oknom určitej veľkosti a pozorovať dopady, zistíme, na ktoré časti obrázka je výstup najviac citlivý.



In [ ]:
attributor = Occlusion(model)

Skúsme túto metódu aplikovať opäť na náš obrázok tukana. Použijeme najprv pomerne malé krycie okno. Ako vidno, model sa evidentne sústredí na tukana, ale vysvietili sa aj iné oblasti – ako napr. niektoré časti stromu.



In [ ]:
img_preproc, img_resized, pred_class = proc_image('data/toucan.png')
attrib = attributor.attribute(img_preproc, target=pred_class,
                              strides = (3, 20, 20),
                              sliding_window_shapes=(3, 30, 30),
                              baselines=0)
visualize_attrib(attrib, outlier_perc=10)

Aby sme ilustrovali vplyv veľkosti krycieho okna, spustíme vizualizáciu ešte raz s väčším oknom. Teraz sa zdá byť najviac relevantná oblasť okolo tukanovho zobáka.



In [ ]:
img_preproc, img_resized, pred_class = proc_image('data/toucan.png')
attrib = attributor.attribute(img_preproc, target=pred_class,
                              strides = (3, 50, 50),
                              sliding_window_shapes=(3, 60, 60),
                              baselines=0)
visualize_attrib(attrib, outlier_perc=10)